In [6]:
# %load combine_sims2.py
import numpy as np
from base.csbase import Combine_Sims_Base
from sklearn.neighbors import NearestNeighbors

""" combine multiple similarities using the weight of each drug/target"""

class Combine_Sims_LimbPerDT(Combine_Sims_Base):
    def __init__(self, k=5):
        self.k = k
        self.copyable_attrs=['k']
    #---------------------------------------------------------------------------------------- 
    
    def combine(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        self._n = Ss.shape[1] # the number of drugs/targets
        self.W = self._compute_weights(Ss, Y) # W: num_sims,n
        S = self._combine_sim(self.W, Ss) 
        return S, self.W
    #---------------------------------------------------------------------------------------- 
    
    def combine_test(self, Ss_te):
        # combine the test similarities, Ss is the test similairties
        self._check_test_sim(Ss_te)
        self._m = Ss_te.shape[1] # the number of test drugs/targets
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros((self._n,self._n)))
        W_te = np.zeros((self._num_sims,self._m), dtype=float) 
        
        for i in range(self._num_sims):
            knn = neigh.kneighbors(1 - Ss_te[i], return_distance=False) # knn.shape = (m,k)
            U = self.W[i,:][knn] # U.shape = (m,k)
            W_te[i,:] = np.mean(U, axis=1)
        
        sum_W_rows = np.sum(W_te, axis=0)
        W_te = W_te/sum_W_rows[None,:] # the sum of each columns in W is 1
        S_te = self._combine_sim(W_te, Ss_te)        
        return S_te, W_te
    #----------------------------------------------------------------------------------------    
    
    def _combine_sim(self, W, Ss):
        # W.shape= num_sims,n, Ss.shape=num_sims,n,n
        W1 = W[:,:,None] # W1.shape = num_sims,n,1
        S = Ss*W1
        S = np.sum(S,axis=0)
        return S
    #---------------------------------------------------------------------------------------- 
    
    def _check_test_sim(self, Ss):
        if Ss.shape[0] != self._num_sims:
            raise RuntimeError("The number of similairties in Ss ({}) is not same with self._num_sims ({})!!".format(Ss.shape[0], self._num_sims))
        if self._n != Ss.shape[2]:
            raise RuntimeError("The self._n:{} is not comparable with Ss's column {}!!".format(self._n, Ss.shape[2]))
    #----------------------------------------------------------------------------------------    
    
    def _compute_weights(self, Ss, Y):
        W = np.zeros((self._num_sims,self._n), dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            C = self._cal_limb(S1, Y, self.k)
            W[i,:] = np.sum(C, axis=1)+1.0/self.k 
            # W[i,h] is the difficulty of d_h in i-th Sim, 1.0/self.k is an smoothing parameter ensuring none zero in W

        W = 1/W  # np.exp(-1*W) has similar performance 
        sum_W_rows = np.sum(W, axis=0)
        W = W/sum_W_rows[None,:] # the sum of each columns in W is 1
        return W
    #---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            for j in range(Y.shape[1]):
                if Y[i,j] == 1: # only consider "1" 
                    C[i,j] = k-np.sum(Y[ii,j])
        C = C/k
        #milb = np.sum(C)/np.sum(Y)
        
        return C
    #---------------------------------------------------------------------------------------- 
#---------------------------------------------------------------------------------------- 


class Combine_Sims_LimbPerDT_1(Combine_Sims_Base):
    def __init__(self, k=5):
        self.k = k
        self.copyable_attrs=['k']
    #---------------------------------------------------------------------------------------- 
    
    def combine(self, Ss, Y):
        self._num_sims = Ss.shape[0]
        self._n = Ss.shape[1] # the number of drugs/targets
        self.W = self._compute_weights(Ss, Y) # W: num_sims,n
        S = self._combine_sim(self.W, Ss) 
        
    
        S[S>=1] = 1.0
        
        return S, self.W
    #---------------------------------------------------------------------------------------- 
    
    def combine_test(self, Ss_te):
        # combine the test similarities, Ss is the test similairties
        self._check_test_sim(Ss_te)
        self._m = Ss_te.shape[1] # the number of test drugs/targets
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros((self._n,self._n)))
        W_te = np.zeros((self._num_sims,self._m), dtype=float) 
        
        for i in range(self._num_sims):
            knn = neigh.kneighbors(1 - Ss_te[i], return_distance=False) # knn.shape = (m,k)
            U = self.W[i,:][knn] # U.shape = (m,k)
            W_te[i,:] = np.mean(U, axis=1)
        
        sum_W_rows = np.sum(W_te, axis=0)
        if np.any(sum_W_rows==0):
            print(W_te)
        W_te = W_te/sum_W_rows[None,:] # the sum of each columns in W is 1
        S_te = self._combine_sim(W_te, Ss_te)
        
        # S_te_sum = np.sum(S_te, axis=1)
        # if np.any(S_te_sum==0):
        #     print(S_te_sum)
        S_te[S_te>1] = 1
        return S_te, W_te
    #----------------------------------------------------------------------------------------    
    
    def _combine_sim(self, W, Ss):
        # W.shape= num_sims,n, Ss.shape=num_sims,n,n
        W1 = W[:,:,None] # W1.shape = num_sims,n,1
        S = Ss*W1
        S = np.sum(S,axis=0)
        return S
    #---------------------------------------------------------------------------------------- 
    
    def _check_test_sim(self, Ss):
        if Ss.shape[0] != self._num_sims:
            raise RuntimeError("The number of similairties in Ss ({}) is not same with self._num_sims ({})!!".format(Ss.shape[0], self._num_sims))
        if self._n != Ss.shape[2]:
            raise RuntimeError("The self._n:{} is not comparable with Ss's column {}!!".format(self._n, Ss.shape[2]))
    #----------------------------------------------------------------------------------------    
    
    def _compute_weights(self, Ss, Y):
        W = np.zeros((self._num_sims,self._n), dtype=float) 
        wg = np.zeros(self._num_sims, dtype=float)  # the global local imbalance based weight
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            milb, C = self._cal_limb(S1, Y, self.k)
            wg[i] = 1- milb
            
            idx1 = np.where(Y==1)
            C[idx1] = 1-C[idx1]
            W[i,:] = np.sum(C, axis=1) #+1.0/(self.k*self._num_sims)
            # W[i,h] is the easiness of d_h in i-th Sim, 1.0/self.k is an smoothing parameter ensuring none zero in W
         
        sum_wg = np.sum(wg)
            
        sum_W_rows = np.sum(W, axis=0)  
        idx0 = np.where(sum_W_rows==0)[0] # indices of durgs whose sum of weight is zero
        sum_W_rows[idx0] = sum_wg
        W[:,idx0] = wg[:,None]
        W = W/sum_W_rows[None,:] # the sum of each columns in W is 1
        return W
    #---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            for j in range(Y.shape[1]):
                if Y[i,j] == 1: # only consider "1" 
                    C[i,j] = k-np.sum(Y[ii,j])
        C = C/k
        milb = np.sum(C)/np.sum(Y)
        
        return milb, C
    #---------------------------------------------------------------------------------------- 
#---------------------------------------------------------------------------------------- 

class Combine_Sims_LimbPerDT_2(Combine_Sims_LimbPerDT_1):
    """ set some samllest weights in each row (of each drug) to zeros"""
    def __init__(self, k=5, rho = 0.6):
        self.k = k
        self.rho = rho # the percetage of weghts of similarities removed
        self.copyable_attrs=['k', 'rho']
    #---------------------------------------------------------------------------------------- 
    
    def combine_test(self, Ss_te):
        # combine the test similarities, Ss is the test similairties
        self._check_test_sim(Ss_te)
        self._m = Ss_te.shape[1] # the number of test drugs/targets
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros((self._n,self._n)))
        W_te = np.zeros((self._num_sims,self._m), dtype=float) 
        
        for i in range(self._num_sims):
            knn = neigh.kneighbors(1 - Ss_te[i], return_distance=False) # knn.shape = (m,k)
            U = self.W[i,:][knn] # U.shape = (m,k)
            W_te[i,:] = np.mean(U, axis=1)
        
        # sum_W_rows = np.sum(W_te, axis=0)
        
        """ !!! No test weights are all zeros"""
        # idx0 = np.where(sum_W_rows==0)[0] # indices of durgs whose sum of weight is zero
        # sum_W_rows[idx0] = self.sum_wg
        # W_te[:,idx0] = self.wg[:,None]       
        # if len(idx0) > 0:
        #     print("weight", idx0)
        
        # set smaller rn_sims weights in each column of W to zero
        if self.rn_sims>0:
            idx_par = np.argpartition(W_te, kth=self.rn_sims, axis=0)
            W_te[idx_par[:self.rn_sims,:],np.arange(W_te.shape[1])[None,:]] = 0   
            
        sum_W_rows = np.sum(W_te, axis=0) # recompute the sum of each columns
        sum_W_rows[sum_W_rows==0] = 1 # ensure no zero vlaues in sum_W_rows, as it will be used as denominator
        W_te = W_te/sum_W_rows[None,:] # the sum of each columns in W is 1

        
        S_te = self._combine_sim(W_te, Ss_te)  
        
        # check if any row of S_te are all 0s
        # sum_S_rows = np.sum(S_te, axis=1)
        # idx0 = np.where(sum_S_rows==0)[0]
        # if len(idx0) > 0:
        #     print("sim", idx0)
        #     S_te[idx0] = np.average(Ss_te[:,idx0,:],axis=0,weights=self.wg)
        S_te[S_te>1] = 1
        return S_te, W_te
    #----------------------------------------------------------------------------------------    
    
    
    # def _compute_weights(self, Ss, Y):
    #     self.rn_sims = int(self.rho*self._num_sims)
        
    #     W = np.zeros((self._num_sims,self._n), dtype=float) 
    #     self.wg = np.zeros(self._num_sims, dtype=float)  # the global local imbalance based weight
    #     for i in range(self._num_sims):
    #         S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
    #         milb, C = self._cal_limb(S1, Y, self.k)
    #         self.wg[i] = 1- milb
            
    #         idx1 = np.where(Y==1)
    #         C[idx1] = 1-C[idx1]
    #         W[i,:] = np.sum(C, axis=1) #+1.0/(self.k*self._num_sims)
    #         # W[i,h] is the easiness of d_h in i-th Sim, 1.0/self.k is an smoothing parameter ensuring none zero in W
        
    #     # set smaller rn_sims weights in each column of W to zero
    #     if self.rn_sims>0:
    #         idx_par = np.argpartition(self.wg, kth=self.rn_sims)
    #         self.wg[idx_par[:self.rn_sims]] = 0
    #     self.sum_wg = np.sum(self.wg)
            
    #     sum_W_rows = np.sum(W, axis=0)  
    #     idx0 = np.where(sum_W_rows==0)[0] # indices of durgs whose sum of weight is zero
    #     sum_W_rows[idx0] = self.sum_wg
    #     W[:,idx0] = self.wg[:,None]
        
    #     # set smaller rn_sims weights in each column of W to zero
    #     if self.rn_sims>0:
    #         idx_par = np.argpartition(W, kth=self.rn_sims, axis=0)
    #         W[idx_par[:self.rn_sims,:],np.arange(W.shape[1])[None,:]] = 0   
    #         sum_W_rows = np.sum(W, axis=0) # recompute the sum of each columns
            
    #     W = W/sum_W_rows[None,:] # the sum of each columns in W is 1
    #     return W
    # #---------------------------------------------------------------------------------------- 
    
    
    def _compute_weights(self, Ss, Y):
        self.rn_sims = int(self.rho*self._num_sims)
        
        W = np.zeros((self._num_sims,self._n), dtype=float) 
        self.wg = np.zeros(self._num_sims, dtype=float)  # the global local imbalance based weight
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            _, C = self._cal_limb(S1, Y, self.k)
            # self.wg[i] = 1- milb
            
            idx1 = np.where(Y==1)
            C[idx1] = 1-C[idx1]
            W[i,:] = np.sum(C, axis=1) #+1.0/(self.k*self._num_sims)
            # W[i,h] is the easiness of d_h in i-th Sim, 1.0/self.k is an smoothing parameter ensuring none zero in W
        
        # set smaller rn_sims weights in each column of W to zero
        # if self.rn_sims>0:
        #     idx_par = np.argpartition(self.wg, kth=self.rn_sims)
        #     self.wg[idx_par[:self.rn_sims]] = 0
        # self.sum_wg = np.sum(self.wg)
            
        wg = np.sum(W, axis=1)
        sum_W_rows = np.sum(W, axis=0)  
        idx0 = np.where(sum_W_rows==0)[0] # indices of durgs whose sum of weight is zero
        # sum_W_rows[idx0] = self.sum_wg
        W[:,idx0] = wg[:,None]
        
        # set smaller rn_sims weights in each column of W to zero
        if self.rn_sims>0:
            idx_par = np.argpartition(W, kth=self.rn_sims, axis=0)
            W[idx_par[:self.rn_sims,:],np.arange(W.shape[1])[None,:]] = 0   
            
        sum_W_rows = np.sum(W, axis=0) # recompute the sum of each columns   
        sum_W_rows[sum_W_rows==0] = 1
        W = W/sum_W_rows[None,:] # the sum of each columns in W is 1
        return W
    #---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            s = S[i,ii]
            z = np.sum(s)
            if z == 0:
                z=1
            C[i] = 1-s@Y[ii,:]/z
        C *= Y #
        milb = np.sum(C)/np.sum(Y)
        
        return milb, C
    #---------------------------------------------------------------------------------------- 
    
#---------------------------------------------------------------------------------------- 



    
    
class Combine_Sims_LimbPerDT2(Combine_Sims_LimbPerDT):
    """ Difference with Combine_Sims_LimbPerDT: considering the influence of similarities in '_cal_limb' function
    Seems slightly good for MFAP but worse for MFAUC, compared with Combine_Sims_Limb3
    """
    
    def combine_test(self, Ss_te): # No change
        # combine the test similarities, Ss is the test similairties
        self._check_test_sim(Ss_te)
        self._m = Ss_te.shape[1] # the number of test drugs/targets
        neigh = NearestNeighbors(n_neighbors=self.k, metric='precomputed')
        neigh.fit(np.zeros((self._n,self._n)))
        W_te = np.zeros((self._num_sims,self._m), dtype=float) 
        
        for i in range(self._num_sims):
            knn = neigh.kneighbors(1 - Ss_te[i], return_distance=False) # knn.shape = (m,k)
            U = self.W[i,:][knn] # U.shape = (m,k)
            W_te[i,:] = np.mean(U, axis=1)
        
        sum_W_rows = np.sum(W_te, axis=0)
        W_te = W_te/sum_W_rows[None,:] # the sum of each columns in W is 1
        S_te = self._combine_sim(W_te, Ss_te)        
        return S_te, W_te
    #----------------------------------------------------------------------------------------     
    
    def _compute_weights(self, Ss, Y):  # No change
        W = np.zeros((self._num_sims,self._n), dtype=float) 
        for i in range(self._num_sims):
            S1 = Ss[i] - np.diag(np.diag(Ss[i])) # set diagnol elements to zeros
            C = self._cal_limb(S1, Y, self.k)
            W[i,:] = np.sum(C, axis=1)+1.0/self.k 
            # W[i,h] is the difficulty of d_h in i-th Sim, 1.0/self.k is an smoothing parameter ensuring none zero in W
      
        W = 1/W
        sum_W_rows = np.sum(W, axis=0)
        W = W/sum_W_rows[None,:] # the sum of each columns in W is 1
        return W
    #---------------------------------------------------------------------------------------- 
    
    def _cal_limb(self, S, Y, k):
        """ S is similarity matrix whose dignoal elememets are zeros"""
        
        neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
        neigh.fit(np.zeros(S.shape))
        knns = neigh.kneighbors(1 - S, return_distance=False)
        
        C = np.zeros(Y.shape, dtype=float)
        for i in range(Y.shape[0]):
            ii = knns[i]
            s = S[i,ii]
            z = np.sum(s)
            if z == 0:
                z=1
            C[i] = 1-s@Y[ii,:]/z
        C *= Y #
        #milb = np.sum(C)/np.sum(Y)
        
        return C
    #---------------------------------------------------------------------------------------- 
#---------------------------------------------------------------------------------------- 

In [63]:
p1=np.array([[1,0.3,0.6,0.4,0.3],
           [0.3,1,0.3,0.1,0.2],
           [0.6,0.3,1,0.4,0.3],
           [0.4,0.1,0.4,1,0],
           [0.3,0.2,0.3,0,1]])
p2=np.array([[1,0.1,0,0.8,0],
           [0.1,1,0.4,0.4,0.1],
           [0,0.4,1,0,0.6],
           [0.5,0.4,0,1,0.2],
           [0,0.1,0.6,0.2,1]])
p3=np.array([[1,0.1,0.8,0.4,0],
           [0.1,1,0.4,0,0],
           [0.8,0.4,1,0,0.8],
           [0.4,0,0,1,0.2],
           [0,0,0.8,0.2,1]])
P=np.array([[1,0,1,1],
          [0,1,0,0],
          [0,0,0,1],
          [1,0,1,0],
          [0,0,0,0]])

In [70]:
Ss=np.array([p1,p2,p3])
csl1 = Combine_Sims_LimbPerDT_2(k=2, rho=1/3)
csl1.combine(Ss,P)

(array([[1.        , 0.18811189, 0.26433566, 0.62377622, 0.13216783],
        [0.1972973 , 1.        , 0.35135135, 0.04864865, 0.0972973 ],
        [0.69090909, 0.34545455, 1.        , 0.21818182, 0.52727273],
        [0.44545455, 0.18181818, 0.        , 1.        , 0.2       ],
        [0.14594595, 0.0972973 , 0.55675676, 0.1027027 , 1.        ]]),
 array([[0.44055944, 0.48648649, 0.54545455, 0.        , 0.48648649],
        [0.55944056, 0.        , 0.        , 0.45454545, 0.        ],
        [0.        , 0.51351351, 0.45454545, 0.54545455, 0.51351351]]))

In [71]:
csl1.combine_test(Ss)

(array([[1.        , 0.1986014 , 0.2958042 , 0.6027972 , 0.1479021 ],
        [0.19783584, 1.        , 0.35108208, 0.04891792, 0.09783584],
        [0.6631068 , 0.3315534 , 1.        , 0.27378641, 0.45776699],
        [0.46502242, 0.26008969, 0.        , 1.        , 0.2       ],
        [0.15479115, 0.1031941 , 0.54201474, 0.0968059 , 1.        ]]),
 array([[0.49300699, 0.48917922, 0.68446602, 0.        , 0.51597052],
        [0.50699301, 0.        , 0.        , 0.65022422, 0.        ],
        [0.        , 0.51082078, 0.31553398, 0.34977578, 0.48402948]]))

In [72]:
csl1._check_test_sim(Ss)

In [73]:
csl1._compute_weights(Ss,P)

array([[0.44055944, 0.48648649, 0.54545455, 0.        , 0.48648649],
       [0.55944056, 0.        , 0.        , 0.45454545, 0.        ],
       [0.        , 0.51351351, 0.45454545, 0.54545455, 0.51351351]])

In [74]:
csl1._combine_sim(csl1._compute_weights(Ss,P),Ss)

array([[1.        , 0.18811189, 0.26433566, 0.62377622, 0.13216783],
       [0.1972973 , 1.        , 0.35135135, 0.04864865, 0.0972973 ],
       [0.69090909, 0.34545455, 1.        , 0.21818182, 0.52727273],
       [0.44545455, 0.18181818, 0.        , 1.        , 0.2       ],
       [0.14594595, 0.0972973 , 0.55675676, 0.1027027 , 1.        ]])

In [75]:
csl1._cal_limb(csl1._combine_sim(csl1._compute_weights(Ss,P),Ss),P,3)

(0.20649213610934108,
 array([[0.14      , 0.        , 0.14      , 0.33037037],
        [0.        , 0.35427574, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.23770492],
        [0.12154696, 0.        , 0.12154696, 0.        ],
        [0.        , 0.        , 0.        , 0.        ]]))

# luo

In [ ]:
S1 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_ddi.txt")
S2 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_disease.txt")
S3 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_se.txt")
S4 = np.loadtxt("./datasets_mv/luo/Dsim/luo_simmat_drugs_tanimoto.txt")
Y_ = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt").T

csl = Combine_Sims_LimbPerDT_2(k=5, rho=0.2)

similarity_matrices = np.array([S1, S2, S3, S4])

similarity_matrices.shape

csl.combine(similarity_matrices,Y_)

csl.combine_test(similarity_matrices)

csl._check_test_sim(similarity_matrices)

csl._compute_weights(similarity_matrices,Y_)

S_FGS_d=csl._combine_sim(csl._compute_weights(similarity_matrices,Y_),similarity_matrices)

In [76]:
np.savetxt('S_FGS_d.txt', S_FGS_d, delimiter='\t')

In [127]:
[S1, S2, S3, S4]

[array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[1.       , 0.19036  , 0.28817  , ..., 0.0028653, 0.32353  ,
         0.3505   ],
        [0.19036  , 1.       , 0.13008  , ..., 0.       , 0.21622  ,
         0.19634  ],
        [0.28817  , 0.13008  , 1.       , ..., 0.0011723, 0.26042  ,
         0.33408  ],
        ...,
        [0.0028653, 0.       , 0.0011723, ..., 1.       , 0.0041667,
         0.0029674],
        [0.32353  , 0.21622  , 0.26042  , ..., 0.0041667, 1.       ,
         0.50394  ],
        [0.3505   , 0.19634  , 0.33408  , ..., 0.0029674, 0.50394  ,
         1.       ]]),
 array([[1.      , 0.044444, 0.11905 , ..., 0.013158, 0.017699, 0.065217],
        [0.044444, 1.      , 0.25926 , ..., 0.066667, 0.0625  , 0.090909],
        [0.11905 , 0.25926 , 1.      , ..., 0.

In [129]:
S2_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_disease.txt")
S1_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_ppi.txt")
S3_p = np.loadtxt("./datasets_mv/luo/Tsim/luo_simmat_proteins_sw-n.txt")

Y_1 = np.loadtxt("./datasets_mv/luo/luo_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p])

similarity_matrices_p.shape

csl_p = Combine_Sims_LimbPerDT_2(k=5, rho=0.7)

csl_p.combine(similarity_matrices_p,Y_1)

csl_p.combine_test(similarity_matrices_p)

csl_p._check_test_sim(similarity_matrices_p)

csl_p._compute_weights(similarity_matrices_p,Y_1)

In [137]:
S_FGS_p=csl_p._combine_sim(csl_p._compute_weights(similarity_matrices_p,Y_1),similarity_matrices_p)

In [143]:
np.savetxt('S_FGS_p.txt', S_FGS_p, delimiter='\t')

In [139]:
[S1_p, S2_p, S3_p]

[array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[1.     , 0.53944, 0.36111, ..., 0.57476, 0.48302, 0.47758],
        [0.53944, 1.     , 0.39005, ..., 0.58358, 0.60386, 0.48448],
        [0.36111, 0.39005, 1.     , ..., 0.50254, 0.35823, 0.4229 ],
        ...,
        [0.57476, 0.58358, 0.50254, ..., 1.     , 0.57068, 0.52829],
        [0.48302, 0.60386, 0.35823, ..., 0.57068, 1.     , 0.46291],
        [0.47758, 0.48448, 0.4229 , ..., 0.52829, 0.46291, 1.     ]]),
 array([[1.  , 0.95, 0.95, ..., 0.95, 0.94, 0.94],
        [0.95, 1.  , 0.95, ..., 0.95, 0.94, 0.94],
        [0.95, 0.95, 1.  , ..., 0.95, 0.94, 0.94],
        ...,
        [0.95, 0.95, 0.95, ..., 1.  , 0.94, 0.94],
        [0.94, 0.94, 0.94, ..., 0.94, 1.  , 1.  ],
        [0.94, 0.94, 0.94, ..., 0.94, 1.  , 1.  ]])]

# e

In [9]:
S1 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_aers-bit.txt")
S2 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_aers-freq.txt")
S3 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_lambda.txt")
S4 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_marginalized.txt")
S5 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_minmaxTanimoto.txt")
S6 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_sider.txt")
S7 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_spectrum.txt")
S8 = np.loadtxt("./datasets_mv/e/Dsim/e_simmat_drugs_tanimoto.txt")

Y_ = np.loadtxt("./datasets_mv/e/e4_admat_dgc.txt").T

csl = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

similarity_matrices = np.array([S1, S2, S3, S4, S5, S6, S7, S8])

similarity_matrices.shape

csl.combine(similarity_matrices,Y_)

csl.combine_test(similarity_matrices)

csl._check_test_sim(similarity_matrices)

csl._compute_weights(similarity_matrices,Y_)

S_FGS_d_E=csl._combine_sim(csl._compute_weights(similarity_matrices,Y_),similarity_matrices)

In [10]:
S_FGS_d_E

array([[1.        , 0.8654838 , 0.84923158, ..., 0.54861172, 0.62799308,
        0.59414295],
       [0.79545455, 1.        , 0.70769231, ..., 0.30645161, 0.55621302,
        0.38857143],
       [0.79196945, 0.73264401, 1.        , ..., 0.3867922 , 0.40369293,
        0.35601814],
       ...,
       [0.64860459, 0.6807629 , 0.70927133, ..., 1.        , 0.77531926,
        0.66024814],
       [0.70079677, 0.76777287, 0.66736847, ..., 0.69497249, 1.        ,
        0.69282502],
       [0.08414577, 0.07629216, 0.0684607 , ..., 0.06778409, 0.08284872,
        1.        ]])

In [11]:
np.savetxt('S_FGS_d_E.txt', S_FGS_d_E, delimiter='\t')

In [18]:
S2_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_go.txt")
S1_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_mismatch-n-k3m1.txt")
S3_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_mismatch-n-k3m2.txt")
S4_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_mismatch-n-k4m1.txt")
S5_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_mismatch-n-k4m2.txt")
S6_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_ppi.txt")
S7_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_spectrum-n-k3.txt")
S8_p = np.loadtxt("./datasets_mv/e/Tsim/e_simmat_proteins_spectrum-n-k4.txt")

Y_1 = np.loadtxt("./datasets_mv/e/e4_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p,S4_p,S5_p,S6_p,S7_p,S8_p])

similarity_matrices_p.shape

Y_1.shape

csl_p = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

csl_p.combine(similarity_matrices_p,Y_1)

csl_p.combine_test(similarity_matrices_p)

csl_p._check_test_sim(similarity_matrices_p)

csl_p._compute_weights(similarity_matrices_p,Y_1)
S_FGS_p_E=csl_p._combine_sim(csl_p._compute_weights(similarity_matrices_p,Y_1),similarity_matrices_p)

(8, 664, 664)

In [23]:
S_FGS_p_E

array([[1.        , 0.42887618, 0.43267835, ..., 0.45043627, 0.46111185,
        0.44752848],
       [0.00310027, 1.        , 0.00650945, ..., 0.00511913, 0.0019992 ,
        0.00258096],
       [0.09337376, 0.19720015, 1.        , ..., 0.2610011 , 0.09157018,
        0.09157018],
       ...,
       [0.09936743, 0.12253065, 0.12447815, ..., 1.        , 0.11564766,
        0.09829708],
       [0.11709816, 0.08851842, 0.09927008, ..., 0.12980966, 1.        ,
        0.12015848],
       [0.24678378, 0.22479193, 0.22612713, ..., 0.24936381, 0.25443237,
        1.        ]])

In [24]:
np.savetxt('S_FGS_p_E.txt', S_FGS_p_E, delimiter='\t')

# gpcr

In [25]:
S1 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_aers-bit.txt")
S2 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_aers-freq.txt")
S3 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_lambda.txt")
S4 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_marginalized.txt")
S5 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_minmaxTanimoto.txt")
S6 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_sider.txt")
S7 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_spectrum.txt")
S8 = np.loadtxt("./datasets_mv/gpcr/Dsim/gpcr_simmat_drugs_tanimoto.txt")

Y_ = np.loadtxt("./datasets_mv/gpcr/gpcr4_admat_dgc.txt").T

csl = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

similarity_matrices = np.array([S1, S2, S3, S4, S5, S6, S7, S8])

similarity_matrices.shape

csl.combine(similarity_matrices,Y_)

csl.combine_test(similarity_matrices)

csl._check_test_sim(similarity_matrices)

csl._compute_weights(similarity_matrices,Y_)

S_FGS_d_gpcr=csl._combine_sim(csl._compute_weights(similarity_matrices,Y_),similarity_matrices)

In [35]:
np.savetxt('S_FGS_d_gpcr.txt', S_FGS_d_gpcr, delimiter='\t')

In [37]:
S2_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_go.txt")
S1_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_mismatch-n-k3m1.txt")
S3_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_mismatch-n-k3m2.txt")
S4_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_mismatch-n-k4m1.txt")
S5_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_mismatch-n-k4m2.txt")
S6_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_ppi.txt")
S7_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_spectrum-n-k3.txt")
S8_p = np.loadtxt("./datasets_mv/gpcr/Tsim/gpcr_simmat_proteins_spectrum-n-k4.txt")

Y_1 = np.loadtxt("./datasets_mv/gpcr/gpcr4_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p,S4_p,S5_p,S6_p,S7_p,S8_p])

similarity_matrices_p.shape

Y_1.shape

csl_p = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

csl_p.combine(similarity_matrices_p,Y_1)

csl_p.combine_test(similarity_matrices_p)

csl_p._check_test_sim(similarity_matrices_p)

csl_p._compute_weights(similarity_matrices_p,Y_1)
S_FGS_p_gpcr=csl_p._combine_sim(csl_p._compute_weights(similarity_matrices_p,Y_1),similarity_matrices_p)

In [38]:
np.savetxt('S_FGS_p_gpcr.txt', S_FGS_p_gpcr, delimiter='\t')

# ic

In [41]:
S1 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_aers-bit.txt")
S2 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_aers-freq.txt")
S3 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_lambda.txt")
S4 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_marginalized.txt")
S5 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_minmaxTanimoto.txt")
S6 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_sider.txt")
S7 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_spectrum.txt")
S8 = np.loadtxt("./datasets_mv/ic/Dsim/ic_simmat_drugs_tanimoto.txt")

Y_ = np.loadtxt("./datasets_mv/ic/ic4_admat_dgc.txt").T

csl = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

similarity_matrices = np.array([S1, S2, S3, S4, S5, S6, S7, S8])

similarity_matrices.shape

csl.combine(similarity_matrices,Y_)

csl.combine_test(similarity_matrices)

csl._check_test_sim(similarity_matrices)

csl._compute_weights(similarity_matrices,Y_)

S_FGS_d_ic=csl._combine_sim(csl._compute_weights(similarity_matrices,Y_),similarity_matrices)

In [42]:
np.savetxt('S_FGS_d_ic.txt', S_FGS_d_ic, delimiter='\t')

In [43]:
S2_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_go.txt")
S1_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_mismatch-n-k3m1.txt")
S3_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_mismatch-n-k3m2.txt")
S4_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_mismatch-n-k4m1.txt")
S5_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_mismatch-n-k4m2.txt")
S6_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_ppi.txt")
S7_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_spectrum-n-k3.txt")
S8_p = np.loadtxt("./datasets_mv/ic/Tsim/ic_simmat_proteins_spectrum-n-k4.txt")

Y_1 = np.loadtxt("./datasets_mv/ic/ic4_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p,S4_p,S5_p,S6_p,S7_p,S8_p])

similarity_matrices_p.shape

Y_1.shape

csl_p = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

csl_p.combine(similarity_matrices_p,Y_1)

csl_p.combine_test(similarity_matrices_p)

csl_p._check_test_sim(similarity_matrices_p)

csl_p._compute_weights(similarity_matrices_p,Y_1)
S_FGS_p_ic=csl_p._combine_sim(csl_p._compute_weights(similarity_matrices_p,Y_1),similarity_matrices_p)

In [44]:
np.savetxt('S_FGS_p_ic.txt', S_FGS_p_ic, delimiter='\t')

# nr

In [45]:
S1 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_aers-bit.txt")
S2 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_aers-freq.txt")
S3 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_lambda.txt")
S4 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_marginalized.txt")
S5 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_minmaxTanimoto.txt")
S6 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_sider.txt")
S7 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_spectrum.txt")
S8 = np.loadtxt("./datasets_mv/nr/Dsim/nr_simmat_drugs_tanimoto.txt")

Y_ = np.loadtxt("./datasets_mv/nr/nr4_admat_dgc.txt").T

csl = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

similarity_matrices = np.array([S1, S2, S3, S4, S5, S6, S7, S8])

similarity_matrices.shape

csl.combine(similarity_matrices,Y_)

csl.combine_test(similarity_matrices)

csl._check_test_sim(similarity_matrices)

csl._compute_weights(similarity_matrices,Y_)

S_FGS_d_nr=csl._combine_sim(csl._compute_weights(similarity_matrices,Y_),similarity_matrices)

In [46]:
np.savetxt('S_FGS_d_nr.txt', S_FGS_d_nr, delimiter='\t')

In [47]:
S2_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_go.txt")
S1_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_mismatch-n-k3m1.txt")
S3_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_mismatch-n-k3m2.txt")
S4_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_mismatch-n-k4m1.txt")
S5_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_mismatch-n-k4m2.txt")
S6_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_ppi.txt")
S7_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_spectrum-n-k3.txt")
S8_p = np.loadtxt("./datasets_mv/nr/Tsim/nr_simmat_proteins_spectrum-n-k4.txt")

Y_1 = np.loadtxt("./datasets_mv/nr/nr4_admat_dgc.txt")

similarity_matrices_p = np.array([S1_p, S2_p,S3_p,S4_p,S5_p,S6_p,S7_p,S8_p])

similarity_matrices_p.shape

Y_1.shape

csl_p = Combine_Sims_LimbPerDT_2(k=5, rho=0.5)

csl_p.combine(similarity_matrices_p,Y_1)

csl_p.combine_test(similarity_matrices_p)

csl_p._check_test_sim(similarity_matrices_p)

csl_p._compute_weights(similarity_matrices_p,Y_1)
S_FGS_p_nr=csl_p._combine_sim(csl_p._compute_weights(similarity_matrices_p,Y_1),similarity_matrices_p)

In [48]:
np.savetxt('S_FGS_p_nr.txt', S_FGS_p_nr, delimiter='\t')